# Far Field - Raw Data

In [ ]:
import h5py
import numpy as np

f = h5py.File("/nfs/chess/raw/2023-3/id3a/berman-3804-a/zx30-t1-ff-0/1/ff/ff1_000231.h5", 'r')
vol=f["imageseries"]["images"][:,:,:] # this is IO bounded
D,H,W=vol.shape
print("Loaded hdf5 raw data", vol.shape,vol.dtype)

In [ ]:
low =np.min(vol,axis=0)
high=np.max(vol,axis=0)

In [ ]:
import plotly.express as px
# high=tifffile.imread('test.tif')
fig = px.imshow(high,zmin=349,zmax=16000, binary_string=True)
fig.update_layout(width=600, height=600, margin=dict(l=0, r=0, b=0, t=0),paper_bgcolor="white",showlegend=False)
fig.update_xaxes(visible=False)
fig.update_yaxes(visible=False)
fig.layout.dragmode='pan'
fig.show(config=dict({'scrollZoom': True}))

# Far Field - Reduced Data


In [ ]:
import os
import numpy as np
import hexrd
import h5py
from pp_dexela import PP_Dexela

# hardcoded
frame_start = 4
threshold  = 50
ostart     = 0. # omega start
oend       = 360.# omega end

filename="/nfs/chess/raw/2023-3/id3a/berman-3804-a/zx30-t1-ff-0/1/ff/ff1_000231.h5"
f = h5py.File(filename, 'r')
# "/nfs/chess/raw/2023-3/id3a/berman-3804-a/zx30-c3-ff/40/ff/ff1_000157.h5", 'r'))
data=f["imageseries"]["images"]
print(data)
D,H,W=data.shape
panel_id=os.path.basename(filename)[0:3];assert panel_id in ["ff1","ff2"]
print(D,H,W,panel_id)

panel_opts = {
    'ff1' : [('add-row', 1944), ('add-column', 1296),('flip', 'v'), ],
    'ff2' : [('add-row', 1944), ('add-column', 1296),('flip', 'h'), ]
}

num_frames = D-frame_start
omw = hexrd.imageseries.omega.OmegaWedges(num_frames)
omw.addwedge(ostart, oend, num_frames)
ppd = PP_Dexela(filename, omw, panel_opts[panel_id], panel_id=panel_id, frame_start=frame_start)
processed = ppd.processed()
processed.metadata['omega']    = ppd.omegas
processed.metadata['panel_id'] = ppd.panel_id.upper()

hexrd.imageseries.write(processed, "__dummy_filename", ppd.PROCFMT, style="npz", threshold=threshold, cache_file=f"test.npz")

In [ ]:
reduced=np.load("test.npz")
# reduced=np.load("/nfs/chess/aux/reduced_data/cycles/2023-3/id3a/berman-3804-a/mg1y-1-ff-0/monolithic_cache_files/mg1y-1-ff-0_5_ff1_000030-cachefile.npz")
D=int(reduced['nframes'])
H,W=reduced['shape']
print(D,H,W)

assert(reduced['dtype'].tobytes().decode('ascii')=="float32")
assert(len(reduced['omega'])==D) # angle ranges from 0 to 360
assert(reduced['panel_id'] in ("FF1","FF2"))

low,high=[np.zeros(shape=(H,W),dtype=np.float32) for I in range(2)]
for Z in range(D):
    # convert the sparse representation to an image
    for r,c,d in zip(reduced[f'{Z}_row'],reduced[f'{Z}_col'],reduced[f'{Z}_data']):
        low[r,c] =np.minimum(d,low [r,c]) if Z else d
        high[r,c]=np.maximum(d,high[r,c]) if Z else d

print(high.dtype,high.shape,np.min(high),np.max(high))

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from ipywidgets import widgets

fig = px.imshow(high,zmin=349,zmax=16000, binary_string=True)
fig.update_layout(width=600, height=600, margin=dict(l=0, r=0, b=0, t=0),paper_bgcolor="white",showlegend=False)
fig.update_xaxes(visible=False)
fig.update_yaxes(visible=False)
fig.show(config=dict({'scrollZoom': True}))